In [4]:
import pandas as pd
import numpy as np
import spacy
import en_core_web_sm


import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams

import sklearn

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk import word_tokenize, pos_tag


import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
all_stopwords = gensim.parsing.preprocessing.STOPWORDS

import scipy as sp

import string
import datetime

from nltk.corpus import stopwords
nltk.download('stopwords')

nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud

from textblob import TextBlob

from gensim import matutils, models
from gensim.models import Phrases
from gensim.models.phrases import Phraser
import gensim.corpora as corpora
import scipy.sparse

from sklearn.feature_extraction import text
import spacy
import en_core_web_sm
import pprint
pp = pprint.PrettyPrinter(indent=4)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [1]:
#mount Drive............
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#link to your folder which contain dataset
import os
os.chdir('/content/drive/My Drive/colab_notebook/Reworked_project')
!ls

 Feb-April.csv			     jan_week3_u.csv
 final_data_project.csv		     jan_week4.csv
 final_data_without_bots.csv	     March_22to28_coronapandemic.csv
'final_Result_aggregated (1).xlsx'   March_22to28_coronavirus.csv
 final_result.csv		     March_22to28_part2.csv
 Jan_final.csv			     New_Data_classified.csv
 jan_week1.csv			     tweet_classifier.ipynb
 jan_week2.csv			     Week5-Week16.csv


In [5]:
#import Sample data.......

new_data = pd.read_csv('Jan_final.csv', lineterminator='\n')
#new_data.text=new_data.text.astype(str)


In [7]:
#checking Columns...............
new_data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'username', 'type', 'id', 'date',
       'retweets', 'text', 'cleaned_tweet'],
      dtype='object')

In [9]:
new_data.shape

(94777, 8)

In [8]:
#dropping cleaned tweet column.....we will create this column again.....
new_data =  new_data.drop('cleaned_tweet', axis = 1)

In [10]:
new_data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'username', 'type', 'id', 'date',
       'retweets', 'text'],
      dtype='object')

In [12]:
final_data = new_data #creating copy to analyze.......

In [13]:
final_data['text']
#text is not clear we have to clean up this text for better analysis......

0        China spurned CDC offer to send team to help c...
1        'Too early' to declare coronavirus outbreak a ...
2        Don’t you hate it when news articles about cor...
3                   This is updated numbers on coronavirus
4        @BorisJohnson #coronavirus uk needs to take pr...
                               ...                        
94772    Google is temporarily closing down its offices...
94773    Ontario Health Minister says “the system is wo...
94774    why don't they invest in finding a cure for th...
94775    @AP @PDChina @ChinaDaily #coronavirus With Chi...
94776    This is good news. Coronavirus basis of many c...
Name: text, Length: 94777, dtype: object

In [14]:
#Running all functions.
#lemmetization
nlp = en_core_web_sm.load(disable=['parser', 'ner']) 

#filtering POS tags
def nouns_adj_verb(text):
    is_noun_adj_verb = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ' or pos[:2] == 'VB' or pos[:2] == 'RB' or pos[:2] == 'NNP' or pos[:2] == 'NNPS' 
    tokenized = word_tokenize(text)
    nouns_adj_verb = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj_verb(pos)] 
    return ' '.join(nouns_adj_verb)

#creating final topic labelling. please do use below code for changing topic 1 to original name......
def new_col(row):
    if row['major_topic'] == 'topic 0':
        val = 'outbreak'
    elif row['major_topic'] == 'topic 1':
        val = 'alternate causes'
    elif row['major_topic'] == 'topic 2':
        val = 'social distancing'
    elif row['major_topic'] == 'topic 3':
        val = 'cleanliness'
    elif row['major_topic'] == 'topic 4':
        val = 'Symptoms'
    elif row['major_topic'] == 'topic 5':
        val = 'Transmission Modes'
    elif row['major_topic'] == 'topic 6':
        val = 'Spread'
    elif row['major_topic'] == 'topic 7':
        val = 'hotspot'
    elif row['major_topic'] == 'topic 8':
        val = 'Death Reports'
    elif row['major_topic'] == 'topic 9':
        val = 'Drug and Medicine'
    elif row['major_topic'] == 'topic 10':
        val = 'Therapies'
    elif row['major_topic'] == 'topic 11':
        val = 'Methods of curing'
    elif row['major_topic'] == 'topic 12':
        val = 'Testing methods'
    elif row['major_topic'] == 'topic 13':
        val = 'Shortage'
    elif row['major_topic'] == 'topic 14':
        val = 'panic buying'
    elif row['major_topic'] == 'topic 15':
        val = 'Market impact'
    elif row['major_topic'] == 'topic 16':
        val = 'Employment'
    elif row['major_topic'] == 'topic 17':
        val = 'business impact'
    elif row['major_topic'] == 'topic 18':
        val = 'Impact Hospital and Clinic'
    elif row['major_topic'] == 'topic 19':
        val = 'Policy Changes'
    elif row['major_topic'] == 'topic 20':
        val = 'Frontline Workers'
    elif row['major_topic'] == 'topic 21':
        val = 'Travel Ban'
    elif row['major_topic'] == 'topic 22':
        val = 'Financial Measures'
    elif row['major_topic'] == 'topic 23':
        val = 'Lockdown'
    elif row['major_topic'] == 'topic 24':
        val = 'Political views'
    elif row['major_topic'] == 'topic 25':
        val = 'racism'
    else:     
        val = 'NaN'
    return val


In [25]:
#running all functions
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag

def nltk_tag_to_wordnet_tag(token_tag):
    if token_tag.startswith('J'):
        return wordnet.ADJ
    elif token_tag.startswith('V'):
        return wordnet.VERB
    elif token_tag.startswith('N'):
        return wordnet.NOUN
    elif token_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_text(text):
    #break text into tokens and find pos tag for each token
    token_tag = nltk.pos_tag(nltk.word_tokenize(text))  
    #tuple of (token, wordnet_tag)
    wordnet_tagger = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), token_tag)
    lemmatized_text = []
    for word, tag in wordnet_tagger:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_text.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_text.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_text)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
#cleaning text function.................................
all_stopwords = gensim.parsing.preprocessing.STOPWORDS
#we identified these stop words manually and thought to include these further to improve analysis. 
all_stopwords_gensim = STOPWORDS.union(set(['mer', 'continue', 'new', '=-PRON-', 'are', 'covid19', 'coronavirus', 'long', 'bring', 'coronatreatment', 'make', 'least', 'watch', 
                                            'show', 'curious', 'city', 'covid19symptoms', 'novel coronaviru', 'com', 'light', 'of', 'surprise', 'know', 'look', 'help', 'may', 
                                            'covid19prevention', 'sar', 'corona treatment', 'review', 'we', 'and', 'via', 'covid19 symptoms', 'bad', 'post', 'able', 'leave', 
                                            'coronavirusitalia', 'basically', 'lot', 'novel', 'top', 'close', 'find', 'due', 's', 'come', 'already', 'tell', 'sure', 'vole', 'real', 
                                            'human', 'much', 'covid19southkorea', 'big', 'youtube', 'ncov coronaviru', 'symptom', 'index', 'last', 'viruse', 'covid19 Italy', 'http', 
                                            'do', 'covid19 treatment', 'possible', 'keep', 'allow', 'like', 'other', 'point', 'Covid-19', 'go', 'way', 'subject', 'put', 'seem', 'Covid19', 
                                            'coronaviru case', 'would', 'coronasymptoms', 'soon', 'one', 'thank', 'need', 'corona pandemic', 'covid19treatment', 'fuck', 'type', 'COVID-19', 
                                            'sample', 'time', 'corona southkorea', 'happen', 'it', 'thread', 'covid19USA', 'relax', 'kill', 'use', 'you', 'year', 'could', 'today', 'day', 
                                            'edu', 'currently', 'call', 'see', 'ncov', 'give', 'number', 'coronapandemic', 'video', ' coronaviru', 'covid19Italy', 'coviditalia', 'covid19 USA',
                                            'perform', 'really', 'been', 'corona', 'coronaviruschina', 'coronavirus', 'back', 'covid19 prevention', 'let', 'hand', 'right', 'virus', 'youtu', 
                                            'get', 'will', 'ask', 'coronaviru ncov', 'from', '-PRON-', 're', 'COVID19', 'people', 'coronavirusoutbreak', 'tuen', 'can', 'information', 'also', 'life', 'open', 'likely', 'to', 'check', 'hope', 'surely', 'far', 'follow', 'ago', 'a;sp', 'feel', 'v', 'take', 'html', 'week', 'corona prevention', 'mail', 'CoronaPandemic', 'amp', 'be', 'many', 'nee', 'wuhancoronaviru', 'say', 'place', 'next', 'hour', 'cause', 'covid-19', 'even', 'case', 'still', 'probabl', 'the', 'in', 'turn', 'map', 'have', 'think', 'yet', 'live', 'travel', 
                                            'covid19 southkorea', 'they', 'u', 'especially', 'word', 'thing', 'corona symptoms', 'belong', 'well', 'coronaprevention', 'newly', 'chance', 
                                            'update', 'covid', 'rule', 'good']))
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@#'

#creating punctuation to remove punctuation........ trial function i used reg ex below to remove punctuations....you can use this as well
def remove_punctuation(text):    
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text


# cleaning master function
#this function will handle links with slashes and without slashes..................
def clean_tweet(tweet):
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    start_index = tweet.find('http') #removing links for specific case of extracted data
    tweet = tweet[0:start_index]
    tweet = re.split(r'http', tweet)[0]
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    tweet = re.sub(r'#\w+','',tweet)
    tweet = re.sub(r'\b\w*(.)\1{2,}\w*\b','', tweet)#for removing words with a character repeting more than 3 times consecutively
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet) #removing words with 2 or less elements
    tweet = lemmatize_text(tweet)
    tweet = re.sub('\s+',' ', tweet) #removing 2 or more consecutive white space
    tweet_token_list = [word for word in tweet.split(' ')
                       if word not in all_stopwords_gensim] # remove stopwords    
    tweet = ' '.join(tweet_token_list)
    return tweet 

In [18]:
#removing duplicate tweets. data had some duplicates values. 
final_data = final_data.drop_duplicates('text', keep = 'first')

In [19]:
final_data.shape

(82373, 8)

In [21]:
#applying cleaning function to text column of data set
final_data['cleaned_text']=final_data.text.apply(clean_tweet)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
#we can see that new column of cleaned text has been created...
final_data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'username', 'type', 'id', 'date',
       'retweets', 'text', 'cleaned_text'],
      dtype='object')

In [23]:
pip install pyLDAvis
#installing visualization library for topic modelling.

     |████████████████████████████████| 1.6MB 2.4MB/s 
     |████████████████████████████████| 552kB 16.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=ade355cb63ee7d876afaf15d5b206540bc72e209fc473b4e903f0dfdaa092001
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=b354c53bcef79eaae7433dc378d319ec22719b5d4d552bf5b264f09939a656f0
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [24]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [30]:
#function for removing stop words......double check to ensure proper cleaning of text.

def stop_remove(tweet):
  tweet_token_list = [word for word in tweet.split(' ')
                       if word not in all_stopwords_gensim] # remove stopwords
  tweet = ' '.join(tweet_token_list)
  return tweet                     

In [29]:
final_data['cleaned_text']

0        china spurn cdc offer send team contain health...
1           early declare outbreak global health emergency
2                          ’ hate news article reach photo
3                                       updated coronaviru
4                                  borisjohnson precaution
                               ...                        
94770    dow worst october negative fear grow cnbc dow ...
94773    ontario health minister “ work ” news conferen...
94774                             invest cure spread world
94775    pdchina chinadaily china population enormous d...
94776                  news basis common cold sars outbrea
Name: cleaned_text, Length: 82373, dtype: object

In [33]:
#function for making tokens of tweet
def tokenizer(text):
  token = word_tokenize(text)
  return token


In [34]:

#creating new column of tokens.................
final_data['token_text']= final_data['cleaned_text'].apply(tokenizer).copy()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
#list of list containing tokens for each tweet
tokens = [[y for y in (re.sub(r'[^A-Za-z0-9]+', '', x) for x in sublst) if y] for sublst in final_data['token_text']]

In [37]:
tokens

[['china',
  'spurn',
  'cdc',
  'offer',
  'send',
  'team',
  'contain',
  'health',
  'secretary'],
 ['early', 'declare', 'outbreak', 'global', 'health', 'emergency'],
 ['hate', 'news', 'article', 'reach', 'photo'],
 ['updated', 'coronaviru'],
 ['borisjohnson', 'precaution'],
 ['glad', 'isn', 'beer'],
 ['china', 'confirms', 'deaths', 'spread', 'multiple', 'country'],
 ['air',
  'canada',
  'temporarily',
  'suspend',
  'flight',
  'beijing',
  'shanghai',
  'amid',
  'concern'],
 ['declares', 'public', 'health', 'emergency'],
 ['drink', 'modelos'],
 ['politician',
  'act',
  'olds',
  'world',
  'brink',
  'wwill',
  'apparently',
  'world',
  'wow',
  'throw',
  'thanos',
  'mcu'],
 [],
 ['worry',
  'infectious',
  'flu',
  'current',
  'mortality',
  'rate',
  'approx',
  'million'],
 ['talk', 'suspect', 'michigan'],
 ['species', 'clam', 'export', 'america'],
 ['dallas',
  'assistant',
  'manager',
  'patient',
  'parkland',
  'memorial',
  'hospita'],
 ['son', 'father', 'bitch'],

In [38]:
clean_text_token = list(tokens)

In [ ]:
#clean_text_token

In [41]:
# Create Dictionary for topic modelling
id2word = corpora.Dictionary(clean_text_token)

# Create Corpus (bag of words for future use)
texts = clean_text_token 

# Term Document Frequency (method of making bag of words, there can be other as well)
corpus = [id2word.doc2bow(text) for text in texts]   #change name for every week

# View
print(corpus[:1])#change name for every week

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]


In [ ]:
id2word

# **Guided LDA hass better approach for topic modelling than LDA. It classify topic based on input topics.**

In [42]:
!pip install guidedlda

     |████████████████████████████████| 2.1MB 2.7MB/s 
  Created wheel for guidedlda: filename=guidedlda-2.0.0.dev22-cp36-cp36m-linux_x86_64.whl size=2479766 sha256=b2142546fee35ce47447d956b10b98350d565150bebb967ba1a14e489e887ae7
  Stored in directory: /root/.cache/pip/wheels/cf/cb/c4/95897cbe4e2e12bd01db3c5cb004c744d2cfd68ecc692ce39e
Successfully built guidedlda


In [43]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet
import numpy as np
np.random.seed(42)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import nltk
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
#same cleaning function except it return tokens required by model instead of full cleared text.
def clean_tweet_2(tweet):
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    tweet = re.sub(r'#\w+','',tweet)
    tweet = re.sub('[^a-zA-Z]',' ',tweet)
    tweet = re.sub(r'\b\w*(.)\1{2,}\w*\b','', tweet)#for removing words with a character repeting more than 3 times consecutively
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet) #removing words with 2 or less elements
    tweet = lemmatize_text(tweet)
    tweet = re.sub('\s+',' ', tweet) #removing 2 or more consecutive white space
    tweet_token_list = [word for word in tweet.split(' ')
                       if word not in all_stopwords_gensim] # remove stopwords

    
    #tweet = ' '.join(tweet_token_list)
    return tweet_token_list

In [45]:
#preparing feature set 
token_vectorizer = CountVectorizer(tokenizer=clean_tweet_2, min_df=4, stop_words=all_stopwords_gensim, ngram_range=(1, 1))
X = token_vectorizer.fit_transform(final_data.cleaned_text)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['', 'aga', 'ame', 'amely', 'ast', 'atter', 'atterly', 'bac', 'bee', 'betwee', 'chec', 'coronapreventio', 'coronaviru', 'coronavirusoutbrea', 'cov', 'deed', 'deta', 'dex', 'dow', 'east', 'eave', 'eed', 'eep', 'eleve', 'ely', 'es', 'eve', 'evertheless', 'ewly', 'ext', 'fee', 'fiftee', 'formatio', 'fuc', 'ght', 'happe', 'hereupo', 'htm', 'huma', 'italy', 'obody', 'ofte', 'ong', 'oone', 'ope', 'othing', 'ove', 'ovel', 'owhere', 'pandemic', 'preventio', 'probab', 'pron', 'rea', 'regard', 'severa', 'soo', 'southkorea', 'terest', 'tha', 'thereupo', 'thic', 'trave', 'treatment', 'tself', 'tue', 'tur', 'umber', 'unt', 'upo', 'usa', 'wee', 'whe', 'whereupo'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [47]:
import guidedlda
tf_feature_names = token_vectorizer.get_feature_names()
word2id=dict((v, idx) for idx, v in enumerate(tf_feature_names))

In [48]:

# # anchor word list designed to tweak the model towards measuring specific topics identified by manual research
#you can select as many as you want depending upon the data
seed_topic_list= [#['chinese', 'wuhan', 'china', 'animal', 'bat', 'eat', 'meat', 'outbreak'],#outbreak
#['bio', 'weapon', 'bioweapon','canada', 'lab','conspiracy'], #alternate causes (bioweapon, conspiracy)
#['work', 'home', 'isolate', 'quarantine', 'outdoor', 'ban', 'distance', 'social', 'gather', 'avoid', 'event', 'cancel'], #Social Distancing
#['alcohol', 'hygiene', 'wash', 'water', 'sanitize', 'sanitizer', 'soap', 'clean'], #Cleanliness
#['mild', 'severe', 'respiratory', 'illness', 'fever', 'difficulty', 'sick', 'cough', 'shortness', 'breath', 'muscle', 'pain', 'headache', 'sore', 'taste', 'smell', 'flu', 'pressure', 'chest', 'inability', 'blue', 'lip', 'urgent'], #Symptoms
#['spread', 'infect', 'air', 'tiny', 'sneeze', 'touch', 'mouth', 'nose', 'aerosol', 'transmit', 'airborne', 'surface', 'exposure'], #Modes of transmission
#['report', 'confirm'], #Spread of cases
#['ship', 'cruise', 'passenger', 'positive', 'board', 'religious', 'service', 'church', 'mosque', 'festival'], #Hotspots and Location
#['deaths', 'die', 'dead'], #Death Reports
#['clinical', 'trial', 'gilead', 'chloroquine','malaria'], #Drugs and Medicines
#['plasma', 'inject', 'herd', 'immunity', 'heat', 'therapy'], #Therapies
#['warm', 'vaccine', 'temperature', 'antibiotic', 'urine', 'lemon'], #Alternate methods of curing
#['test', 'kit', 'testing'], #Testing Methods
['shortage'], #Shortage of Products
['toilet', 'buy', 'toilet', 'paper', 'grocery', 'panic'], #Panic Buying
['dow','stock', 'market', 'oil', 'tax', 'recession', 'economy', 'crash', 'collapse', 'depression', 'decline', 'fall'], #Stock Markets
['job', 'unemployment'], #Employment
['factory', 'industry', 'company', 'business', 'finance', 'shutdown', 'loss', 'supply', 'chain', 'loose', 'profit', 'impact', 'closure']]#, #Impact on Business
#['hospital', 'bed', 'guidance', 'guide', 'clinic','ppe'], #Impact on Hospital and Clinics
#['treatment', 'health', 'policy', 'safety', 'regulation'], #Policy changes regarding treatments
#['community', 'frontline', 'worker', 'doctor', 'nurse', 'staff'], #Frontline Workers 
#['air', 'border', 'cancel', 'flight', 'ban', 'transport', 'airline'], #Travel Restrictions
#['refund', 'relief', 'availability', 'allocate', 'relief', 'package', 'claim', 'payment', 'pension'], #Financial Measures
#['stay', 'order', 'mandate', 'restriction', 'lockdown', 'government', 'announce', 'shut', 'require'], #Lockdown Regulations
#['politics', 'biden', 'vote', 'elect', 'democrat', 'republican', 'bernie', 'election', 'brexit'], #Political Views
#['chinese', 'asian', 'racist', 'racism', 'anti', 'sentiment', 'fear', 'xenophobic']] #Racism

In [49]:
#model fitting
model = guidedlda.GuidedLDA(n_topics=5, n_iter=100, random_state=7, refresh=10)
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id
model.fit(X, seed_topics=seed_topics, seed_confidence=1)

/usr/local/lib/python3.6/dist-packages/guidedlda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):


In [50]:
n_top_words = 15
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(tf_feature_names)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: china wuhan  spread confirm chinese outbreak report death sars health deadly infect news pneumonia
Topic 1: china chinese coronaviru spread wuhan flu outbreak mask health die stop stay world shit want
Topic 2: china outbreak market spread wuhan chinese global death fear stock world health sars animal emergency
Topic 3: china confirm health wuhan report outbreak chinese spread official cdc airport state screen emergency death
Topic 4: china spread wuhan chinese coronaviru outbreak flu vaccine sars news world work die disease trump


In [52]:
#NLP
import sklearn

#plotting
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

In [53]:
#visualization to check inter topic distance.......we see that topics are clearly seperated.....
panel = pyLDAvis.sklearn.prepare(model, X, token_vectorizer, mds='tsne')
panel

/usr/local/lib/python3.6/dist-packages/guidedlda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      150.222153  -72.269554       1        1  26.954586
1       51.163364 -159.968018       2        1  20.267999
4      -73.144554   21.523815       3        1  18.913526
2      -66.410133 -110.606438       4        1  17.526477
0       54.417324   12.443444       5        1  16.337412, topic_info=         Term          Freq         Total Category  logprob  loglift
0               1915.000000   1915.000000  Default  30.0000  30.0000
1720  confirm   6674.000000   6674.000000  Default  29.0000  29.0000
5123   market   1430.000000   1430.000000  Default  28.0000  28.0000
1397    china  20009.000000  20009.000000  Default  27.0000  27.0000
3860   health   6719.000000   6719.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
7983    state    405.061636   2311.156165   Topic5  -5.3888   0.0702
9391    world    407.894169   2696.078881   Topic5  -5.3819  -0.0768
7407   screen    368.238709   1783.022131   Topic5  -5.4841   0.2344
1268      cdc    377.680485   2263.624017   Topic5  -5.4588   0.0210
6170  patient    367.294532   1977.750638   Topic5  -5.4867   0.1282

[470 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         2  0.001044        
0         5  0.999000        
4         4  0.979181    aapl
26        2  0.924852     abt
26        5  0.083000     abt
...     ...       ...     ...
9444      3  0.099229   wuhan
9444      4  0.134593   wuhan
9444      5  0.236039   wuhan
9543      2  0.988825    ysol
9588      1  1.000916  zraick

[947 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 5, 3, 1])

In [54]:
doc_topic = model.transform(X)
print(doc_topic)

/usr/local/lib/python3.6/dist-packages/guidedlda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):


[[1.92343370e-03 1.30371172e-03 2.36776768e-03 9.90755199e-01
  3.64988774e-03]
 [1.59130639e-03 7.30020210e-04 2.35653771e-01 7.61400451e-01
  6.24451601e-04]
 [1.99325926e-01 4.39760983e-02 8.17768057e-02 1.10755345e-02
  6.63845636e-01]
 ...
 [1.01134723e-02 6.68599055e-03 9.65533020e-03 4.15124798e-03
  9.69393959e-01]
 [8.83465634e-01 1.09348619e-03 1.11976359e-01 8.55739830e-04
  2.60878082e-03]
 [3.64948027e-01 5.26998274e-02 4.88853647e-01 6.41486245e-04
  9.28570120e-02]]


In [56]:
#making data frame of each topic score for a tweet.
columns_label = ['topic {}'.format(i) for i in range(5)]  # number of topics
topic_vector = pd.DataFrame(doc_topic, columns = columns_label)#dataframe of doc-topics
tp=topic_vector.round(2)
dominant_topic = np.argmax(tp.values, axis=1)
tp['dominant_topic'] = dominant_topic


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:2145: RuntimeWarning: overflow encountered in multiply
  result = com.values_from_object(self).round(decimals)


In [57]:
tp['tweet']=list(final_data['text'])


In [58]:
tp.isnull().sum() #no missing value just to check

topic 0           0
topic 1           0
topic 2           0
topic 3           0
topic 4           0
dominant_topic    0
tweet             0
dtype: int64

In [59]:
tp.columns

Index(['topic 0', 'topic 1', 'topic 2', 'topic 3', 'topic 4', 'dominant_topic',
       'tweet'],
      dtype='object')

**SENTIMENT ANALYSIS...................................**

In [60]:
# starting sentiment Analysis of tweets............................
!pip install vaderSentiment

     |████████████████████████████████| 133kB 2.7MB/s 


In [62]:
#importing text into a list
corpus = list(final_data['cleaned_text'])

In [63]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment_machine = SentimentIntensityAnalyzer()

# compute sentiment scores (polarity) and labels
sentiment_scores_vad = [sentiment_machine.polarity_scores(doc) for doc in corpus]

In [64]:
#making data frame of VADER scores and checking Distribution of compund scores.
result_vad = pd.DataFrame(sentiment_scores_vad)
result_vad.compound.describe()

count    82373.000000
mean        -0.100688
std          0.376787
min         -0.994400
25%         -0.381800
50%          0.000000
75%          0.000000
max          0.987200
Name: compound, dtype: float64

In [65]:
#storing compound values to a list......and labeling sentiment as per range
compound_score= list(result_vad['compound'])                        
sentiment_category_vad = ['positive' if score > 0.05 
                             else 'negative' if score < -.05 
                                 else 'neutral' 
                                     for score in compound_score]

In [66]:
# sentiment statistics per rating

#creating data frame of these results and checking correlation with ratings....
sen_jan = pd.DataFrame([list(final_data['cleaned_text']), sentiment_category_vad, list(result_vad['compound'])]).T
sen_jan.columns = ['clean_tweet','sentiment_category_vad','compound_Score']
sen_jan['compound_Score'] = sen_jan.compound_Score.astype('float')

In [67]:
sen_jan['sentiment_category_vad'].value_counts() #distribution of sentiment

neutral     34642
negative    31254
positive    16477
Name: sentiment_category_vad, dtype: int64

In [68]:
#appending results in our final data frame
tp['sentiments_score']=sen_jan['compound_Score']
tp['sentiment']= sen_jan['sentiment_category_vad']

In [69]:
tp.columns

Index(['topic 0', 'topic 1', 'topic 2', 'topic 3', 'topic 4', 'dominant_topic',
       'tweet', 'sentiments_score', 'sentiment'],
      dtype='object')

In [70]:
tp.isnull().sum()

topic 0             0
topic 1             0
topic 2             0
topic 3             0
topic 4             0
dominant_topic      0
tweet               0
sentiments_score    0
sentiment           0
dtype: int64

In [71]:
topic_vector.head()

,topic 0,topic 1,topic 2,topic 3,topic 4
0,0.001923,0.001304,0.002368,0.990755,0.003650
1,0.001591,0.000730,0.235654,0.761400,0.000624
2,0.199326,0.043976,0.081777,0.011076,0.663846
3,0.155217,0.355536,0.139550,0.061552,0.288145
4,0.004845,0.035235,0.001119,0.949393,0.009409


In [72]:
#creating a list of top topics (if the top topic has a weight less than 0.2, the output will be NA)
topic_top = []
for i in range(len(topic_vector)):
  for top, val in topic_vector.iloc[i,].items():
    if val == max(topic_vector.iloc[i,]):
      if val >0.2:
        t = top
      else:
        t = "NaN"
      topic_top.append(t)
      break
#Creating tweet - top topic data frame
df_doc_topic = pd.DataFrame({'major topic': topic_top}) #this gives out that which tweet has been classified in which topic.
df_doc_topic['month']='jan'
df_doc_topic

,major topic,month
0,topic 3,jan
1,topic 3,jan
2,topic 4,jan
3,topic 1,jan
4,topic 3,jan
...,...,...
82368,topic 2,jan
82369,topic 4,jan
82370,topic 4,jan
82371,topic 0,jan


In [73]:
df_doc_topic['major topic'].value_counts()

topic 3    21400
topic 1    19024
topic 4    15064
topic 0    14299
topic 2    12380
NaN          206
Name: major topic, dtype: int64

In [76]:
tp['major_topic']= df_doc_topic['major topic']

In [77]:
tp.head()

,topic 0,topic 1,topic 2,topic 3,topic 4,dominant_topic,tweet,sentiments_score,sentiment,major_topic
0,0.00,0.00,0.00,0.99,0.00,3,China spurned CDC offer to send team to help c...,0.0000,neutral,topic 3
1,0.00,0.00,0.24,0.76,0.00,3,'Too early' to declare coronavirus outbreak a ...,-0.3818,negative,topic 3
2,0.20,0.04,0.08,0.01,0.66,4,Don’t you hate it when news articles about cor...,-0.5574,negative,topic 4
3,0.16,0.36,0.14,0.06,0.29,1,This is updated numbers on coronavirus,0.0000,neutral,topic 1
4,0.00,0.04,0.00,0.95,0.01,3,@BorisJohnson #coronavirus uk needs to take pr...,0.0000,neutral,topic 3


In [ ]:
tp.to_csv("jan_analysis.csv")